In [ ]:
#load the data set

In [ ]:
cab<-read.csv("Cab_Data.csv")
cus_id<-read.csv("Customer_ID.csv")
city<-read.csv("City.csv")
transaction<-read.csv("Transaction_ID.csv")

In [ ]:
#check how many people in each gender
gender<-cus_id$Gender
table(gender)

There are 22609 females and 26562 males.

In [ ]:
#Calculate the profit for each order
cab$profit<-cab$Price.Charged-cab$Cost.of.Trip
#merge two tables together and delete the missing value
cab <- merge(cab, transaction, by = "Transaction.ID", all = FALSE)

There are some ansacions that haven't been colleced in "cab" file, I choose to ignore them and delete them.

Reason: In "transaction" file, only customer id and payment method are collected, while no specific amount of KM and costs are collected in "cab" file

In [ ]:
library(dplyr)
customer_info <- merge(customer_info, cus_id, by="Customer.ID")
#set income_status based on their income and separate them into 3 groups
customer_info$income_status <- cut(customer_info$Income..USD.Month., 
                        breaks = c(-Inf, 3000, 15000, Inf), 
                        labels = c(1, 2, 3),
                        include.lowest = TRUE)
#Calculate the total profit and mean profit based on cities
company_city_av<-cab %>%
  group_by(Company,City) %>%
  summarize(total_profit = sum(profit),
            average_profit = mean(profit))

#Calculate the total KM based on cities
company_city_road<-cab %>%
  group_by(Company,City) %>%
  summarize(total_road = sum(KM.Travelled),
            average_road = mean(KM.Travelled))
company_city <- company_city_av
company_city$total_road <- company_city_road$total_road

#Calculate how much profit they can get per KM
company_city$profit_per_KM <- company_city$total_profit/company_city$total_road
cus_id$income_status <- cut(cus_id$Income..USD.Month., 
                                   breaks = c(-Inf, 3000, 15000, Inf), 
                                   labels = c(1, 2, 3),
                                   include.lowest = TRUE)
  
cab <- merge(cab,cus_id,by="Customer.ID")
#calculate the total income in each group of income status
company_incomestatus <- cab %>%
  group_by(Company, income_status) %>%
  summarize(total_profit_income=sum(profit))

#Calculate how many customers in each income status in each company
company_income_percentage <- customer_info %>%
  group_by(Company,income_status) %>%
  summarise(No.income = n())

#Calculate how many customers in each gender in each company
company_gender_percentage <- customer_info %>%
  group_by(Company,Gender) %>%
  summarise(No.Gender = n())

#Calculate the percenage of each gender group 
company_gender_percentage <- company_gender_percentage %>%
  group_by(Company) %>%
  mutate(total = sum(No.Gender)) %>%
  group_by(Company, Gender) %>%
  summarize(percent = sum(No.Gender) / first(total) * 100)

In [ ]:
library(ggplot2)
ggplot(company_gender_percentage, aes(x = Company, y = percent, fill = Gender)) +
  geom_bar(stat = "identity", position = "fill") +
  geom_text(aes(label = scales::percent(percent/100), group = Gender),
            position = position_fill(vjust = 0.5), 
            color = "white", size = 3) +
  scale_fill_manual(values = c("orange", "darkblue")) + 
  labs(title = "Percentage of Gender", x = "Company", y = "Percentage of Gender")


In [ ]:
#Calculate the percentage of each income status
company_percentage <- company_percentage %>%
  group_by(Company) %>%
  mutate(total = sum(No.income)) %>%
  group_by(Company, income_status) %>%
  summarize(percent = sum(No.income) / first(total) * 100)

#Calculate how many profits he can get in each income status
compnay_incomestatus<- company_incomestatus %>%
  summarise(total = sum(total_profit_income)) %>%
  ungroup()

library(ggplot2)
company_incomestatus_large <- company_incomestatus %>%
  group_by(Company) %>%
  mutate(total_profit_income_max = max(total_profit_income)) %>%
  filter(total_profit_income >= 0.1 * total_profit_income_max) %>%
  ungroup()

income_status_colors <- c("1" = "pink", "2" = "lightgreen", "3" = "lightblue")
income_status_labels <- c("1" = "<$3000", "2" = "$3000~$15000", "3" = ">$15000")
ggplot(company_incomestatus, aes(x = Company, y = total_profit_income, fill = income_status)) +
  geom_bar(stat = "identity", position = "stack") +
  scale_fill_manual(values = income_status_colors, labels = income_status_labels) +
  labs(title = "Total Income",
       x = "Company",
       y = "Total Income",
       fill = "Income Status") +

  geom_text(data = company_incomestatus_large, aes(label = total_profit_income), 
              position = position_stack(vjust = 0.5), 
              size = 3)

#Calculate the percentage of each income status
company_income_percentage <- company_income_percentage %>%
  group_by(Company) %>%
  mutate(total = sum(No.income)) %>%
  group_by(Company, income_status) %>%
  summarize(percent = sum(No.income) / first(total) * 100)

ggplot(company_income_percentage, aes(x = Company, y = percent, fill = income_status)) +
  geom_bar(stat = "identity", position = "fill") +
  geom_text(aes(label = scales::percent(percent/100), group = income_status),
            position = position_fill(vjust = 0.5), 
            color = "white", size = 3) +
  scale_fill_manual(values = c("orange", "darkblue","darkgreen"),, labels = income_status_labels) + 
  labs(title = "Percentage of Income Status", x = "Company", y = "Percentage of Income Status")


In [ ]:
#Calculate the total profit of each gender
company_gender <- cab %>%
  group_by(Company, Gender) %>%
  summarize(total_profit_income=sum(profit))

gender_colors <- c("Female" = "pink", "Male" = "lightblue")
ggplot(company_gender, aes(x = Company, y = total_profit_income, fill = Gender)) +
  geom_bar(stat = "identity", position = "stack") +
  scale_fill_manual(values = gender_colors) +
  labs(title = "Total Income by Gender",
       x = "Company",
       y = "Total Income",
       fill = "Gender") +
  geom_text(aes(label = total_profit_income), 
            position = position_stack(vjust = 0.5), 
            size = 3)

In [ ]:
#Divide the customers into 4 groups based on age
customer_info$agegroup <- cut(customer_info$Age, 
                    breaks = c(18, 25, 35, 60, Inf), 
                    labels = c(1, 2, 3, 4),
                            include.lowest = TRUE)
#Calculate the total profit in each group of ages
company_age <- cab %>%
  group_by(Company, agegroup) %>%
  summarize(total_profit_income=sum(profit))

cab$agegroup <- cut(cab$Age, 
                    breaks = c(18, 25, 35, 60, Inf), 
                    labels = c(1, 2, 3, 4),
                    include.lowest = TRUE)

#Calculate the number of cusomers in each group of ages
company_agegroup_percentage <- customer_info %>%
  group_by(Company,agegroup) %>%
  summarise(No.age = n())

#calculate the percentage of each group of ages
company_agegroup_percentage <- company_agegroup_percentage %>%
  group_by(Company) %>%
  mutate(total = sum(No.age)) %>%
  group_by(Company, agegroup) %>%
  summarize(percent = sum(No.age) / first(total) * 100)

In [ ]:
agegroup_colors <- c("1" = "pink", "2" = "lightgreen", "3" = "lightblue", "4" = "yellow")
agegroup_labels <- c("1" = "18-25", "2" = "26-35", "3" = "36-60", "4" = "60+")
ggplot(company_agegroup_percentage, aes(x = Company, y = percent, fill = agegroup)) +
  geom_bar(stat = "identity", position = "fill") +
  geom_text(aes(label = scales::percent(percent/100), group = agegroup),
            position = position_fill(vjust = 0.5), 
            color = "black", size = 3) +
  scale_fill_manual(values = agegroup_colors, labels = agegroup_labels) + 
  labs(title = "Percentage of agegroup", x = "Company", y = "Percentage of agegroup")

company_age_large <- company_age %>%
  group_by(Company) %>%
  mutate(total_profit_income_max = max(total_profit_income)) %>%
  filter(total_profit_income >= 0.1 * total_profit_income_max) %>%
  ungroup()

ggplot(company_age, aes(x = Company, y = total_profit_income, fill = agegroup)) +
  geom_bar(stat = "identity", position = "stack") +
  scale_fill_manual(values = agegroup_colors, labels = agegroup_labels) +
  labs(title = "Total Income",
       x = "Company",
       y = "Total Income",
       fill = "agegroup") +
  geom_text(data = company_age_large, aes(label = total_profit_income), 
              position = position_stack(vjust = 0.5), 
              size = 2)

ggplot(company_city, aes(x = City, y = profit_per_KM, fill = Company)) +
  geom_bar(stat = "identity", position = "dodge", alpha = 0.7) +
  labs(title = "Profit per KM by City",
       x = "City",
       y = "Profit per KM",
       fill = "Company") +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=0.4))

In [ ]:
#Calculate the number of orders in each city
orders_city <- cab %>%
  group_by(City, Company) %>%
  summarise(No.Orders = n())

ggplot(orders_city, aes(x = City, y = No.Orders, fill = Company)) +
  geom_bar(stat = "identity", position = "dodge", alpha = 0.7) +
  labs(title = "No. of Orders per city",
       x = "City",
       y = "No. of orders",
       fill = "Company") +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=0.4))

#Calculate the number of customers of each company in each city
customers_city <- cab %>%
  group_by(City, Company) %>%
  summarise(No.customers = n_distinct(Customer.ID))
customers_city <- merge(customers_city, city, by = "City")

Assumptions:
1. Outliers are shown in "cab" for missing specific order information, and the only information we can get is the payment method. Hence I choose to delete these outliers in the "cab" file.
2. Due to different amount of orders in each city and each company, and different amount of KM, profit per KM is calculated and used to compare in each city and company.
3. Due to huge gap of number of customers in each company, the percentage of age, income status etc. are calculated to show the structure of customers.
4. Since the total number of customers belong to these two companies is not equal to the total users, I assume that there are other companies exist but not being collected into these files.
5. In order to analyze conveniently, I combine "transaction","cus_id" and "city" with "cab". Moreover, I create several data sets to calculate the total profit of each group, such as age group, income_status etc.,and the percentage of each group. 

Conclusion:
1. Each company has a similar customer structure, with nearly the same proportion of age, gender and income structure.
2. Yellow cab company started early, compared with pink cab company, it has more abundant customer resources and reserves, and has a high influence in various cities. 
3. Yellow cab company has far more orders than pink cab company, and the profit per km is much higher in most of the cities. 
4. Yellow cab company has a strong presence in large cities and are more established than pink cab company.
Therefore, for the above reasons, yellow cab company has less investment risk and greater profit potential. Consequentially, I suggest to invest yellow cab company.
